# 실습목표

### 1) 서울시 구별 인구수와 CCTV 설치 비율에 대한 분석을 위해 공공데이터를 확인하고 적절한 방법으로 처리하시오. 
### 2) 최근 3년간 CCTV증가율과 한국인, 외국인, 고령자의 비율에 따른 데이터 변화를 파악하시오.

# 0. 데이터 준비

### 서울시 공공데이터 다운 받기

#### 1. 서울시 자치구 년도별 CCTV 설치 현황 다운

* https://data.seoul.go.kr/dataList/datasetList.do 

* 서울 열린데이터 광장 → 서울시 자치구 년도별 CCTV 설치현황 검색 → 데이터 다운 (PC 다운시 파일명은 “서울시cctv”로 저장)

#### 2. 서울시 주민등록인구(구별) 통계 자료 다운

* http://data.seoul.go.kr/dataList/419/S/2/datasetView.do

* 서울 열린데이터 광장 → 서울시 주민등록인구(구별) 통계 → 데이터 다운(TXT) (PC 다운시 파일명은 “서울인구통계”로 저장)

# 1. 데이터 불러오고 구조파악하기

### [서울인구통계] 탭으로 분리된(tsv) .txt 텍스트파일 불러오기

In [ ]:
import pandas as pd
import numpy as np

# data = pd.read_csv("파일경로", sep = "\t", , engine="python", encoding = "인코딩방식")
pop_seoul = pd.read_csv("서울인구통계.txt",sep="\t", engine="python", thousands = ",", encoding = "utf-8")
pop_seoul.head()

### [서울인구통계] 자치구별 인구합계, 한국인 합계, 외국인 합계, 고령자 데이터만을 추출
#### index 0,1행은 중복으로 필요 없는 행으로 삭제

In [ ]:
pop1=pop_seoul[["자치구","인구","인구.3","인구.6","65세이상고령자"]]
pop_seoul=pop1.drop([0,1])
pop_seoul.head()

### [서울인구통계] 컬럼명을 구분할 수 있도록 리네임

In [ ]:
pop_seoul.rename(columns={
    pop_seoul.columns[0]:"구별",
    pop_seoul.columns[1]:"인구수",
    pop_seoul.columns[2]:"한국인",
    pop_seoul.columns[3]:"외국인",
    pop_seoul.columns[4]:"고령자"},inplace=True)
pop_seoul.head()

### [서울시 CCTV] 엑셀파일을 열고 5번째까지 자료를 읽기

In [ ]:
cctv_seoul=pd.read_excel("서울시cctv.xlsx")
cctv_seoul.head()

### [서울시 CCTV] 컬럼의 데이터를 확인

In [ ]:
cctv_seoul.columns

### [서울시 CCTV] 인구통계 자료와 통일을 하기 위해 기관명을 구별로 인덱스 이름 변경

In [ ]:
cctv_seoul.rename(columns={cctv_seoul.columns[0] : "구별"}, inplace=True)
cctv_seoul.rename(columns={cctv_seoul.columns[1] : "cctv수"}, inplace=True) # inplace=TRUE로 해야 값까지 다 포함해서 가져감
cctv_seoul.head() # 소계를 cctv수로 이름 변경

# 2. cctv 데이터 파악

### sort_values : 값을 기준으로 정렬, by=기준 열

In [ ]:
cctv_seoul.sort_values(by="cctv수", ascending=True).head(10) #True 오름차순
#cctv수가 가장 작은 지역은 도봉구, 중랑구, 강북구 순으로 나타남

In [ ]:
cctv_seoul.sort_values(by="cctv수", ascending=False).head(10) #False 내림차순
#cctv수가 가장 많은 지역은 강남구, 관악구, 구로구 순으로 나타남

### NaN 값 확인

In [ ]:
cctv_seoul.isnull().sum()  #NaN 값이 있는 개수를 확인 가능
cctv_seoul.loc[cctv_seoul.isnull()["2011년 이전"]] #2011년 이전 열을 기준으로 NaN 값 추출
#동대문구, 중랑구에서 2013년까지의 데이터가 누락되어 있음. 데이터 유지, 삭제, 대체값 결정

### 최근 3년간 cctv 증가율 비교
#### 증가율 = 2016~2018 cctv 수/2016년 이전 cctv 수

In [ ]:
cctv_seoul["최근증가율"]=(cctv_seoul["2018년"]+cctv_seoul["2017년"]+cctv_seoul["2016년"])/(cctv_seoul["cctv수"]-(cctv_seoul["2018년"]+cctv_seoul["2017년"]+cctv_seoul["2016년"]))*100
cctv_seoul.sort_values(by="최근증가율",ascending=False).head(5)

#최근 3년간 CCTV가 그 이전 대비 많이 증가한 구는 '중구', '영등포구', '성동구', '금전구', '광진구'라는 것도 알 수 있다.

# 3. 서울시 인구 데이터 파악

### 인구 데이터 확인

In [ ]:
pop_seoul.head()

### index 2의 행 삭제를 위해 [2] 입력 

In [ ]:
pop_seoul.drop([2], inplace=True)
pop_seoul.head()

### 데이터 타입 확인. object로 연산이 불가 하기 때문에 int, float형으로 변경 필요

In [ ]:
pop_seoul.dtypes
pop_seoul=pop_seoul.astype({"인구수":"int","한국인":"int","외국인":"int","고령자":"int"})

### 구별 '외국인비율'과 '고령자비율'을 계산

In [ ]:
pop_seoul["외국인 비율"]=pop_seoul["외국인"]/pop_seoul["인구수"]*100
pop_seoul["고령자 비율"]=pop_seoul["고령자"]/pop_seoul["인구수"]*100
pop_seoul.head()

#외국인비율과 고령자비율이 마지막열에 추가됨

### 전체 '외국인비율', '외국인'으로 정렬

In [ ]:
pop_seoul.sort_values(by="외국인 비율", ascending=False).head()
#외국인비율이 높은 구는 영등포구, 중구, 금천구, 구로구, 용산구 순으로 나타나고 있다.

In [ ]:
pop_seoul.sort_values(by="고령자 비율", ascending=False).head()
#고령자 비율이 높은 구는 강북구, 도봉구, 중구, 종로구, 은평구 순으로 나타나고 있다

# 4. CCTV 데이터와 인구 데이터 합치고 분석

### cctv_seoul의 '구별' 열의 내용이 글자사이 공백으로 인해 병합이 되지 않기 때문에 rename 실시

In [ ]:
cctv_seoul["구별"]=["강남구","강동구","강북구","강서구","관악구","광진구","구로구","금천구","노원구","도봉구",
"동대문구","동작구","마포구","서대문구","서초구","성동구","성북구","송파구","양천구","영등포구","용산구","은평구",
"종로구","중구","중랑구"]

### merge를 사용하여 병합

In [ ]:
result = pd.merge(cctv_seoul, pop_seoul, on="구별")
result

#구별 열을 기준으로 cctv_seoul과 pop_seoul을 합친 데이터 생성

### 의미없는 컬럼을 del 명령어로 제거
### 일반적으로 행을 삭제할때는 drop, 열을 삭제할때는 del을 사용
### 2011~18년까지의 cctv수는 의미가 없으므로 삭제

In [ ]:
del result["2011년 이전"]
del result["2012년"]
del result["2013년"]
del result["2014년"]
del result["2015년"]
del result["2016년"]
del result["2017년"]
del result["2018년"]
result.head()

In [ ]:
# 외국인 비율에 따른 CCTV 최근 증가율 관계를 보기 위해 산점도 출력
import seaborn as sb
result.rename(columns = {"외국인 비율" : "foreigner ratio"}, inplace = True)
result.rename(columns = {"최근증가율" : "cctv growth ratio"}, inplace = True)
sb.scatterplot(x="foreigner ratio", y="cctv growth ratio", data=result)